## In this notebook, we'll turn the most useful code cells from notebook 04_custom_datasets into a series of Python scripts saved to a directory called going_modular.

In [1]:
from pathlib import Path

# Create the 'going_modular' directory if it doesn't exist
Path("going_modular").mkdir(exist_ok=True)

In [2]:
# get_data.py

%%writefile going_modular/get_data.py
"""
Contains functionality for creating data folders and
downloading data.
"""
import os
import requests
import zipfile
from pathlib import Path

# Set up path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
  print(f"{image_path} directory exists.")
else:
  print(f"Did not find {image_path} directory, creating one...")
  image_path.mkdir(parents=True, exist_ok=True)

  # Download images
  with open(data_path / "pizza_steak_sushi_20_percent.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

  # Unzip image data
  with zipfile.ZipFile(data_path / "pizza_steak_sushi_20_percent.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

  # Remove zip file
  os.remove(data_path / "pizza_steak_sushi_20_percent.zip")

Writing going_modular/get_data.py


In [3]:
# Get images
!python going_modular/get_data.py

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


In [4]:
# data_setup.py

%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create datasets
  train_data = datasets.ImageFolder(train_dir, transform=transform)

  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,  # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [5]:
!ls

data  going_modular  sample_data


In [6]:
image_path = Path("data/pizza_steak_sushi")
train_data_path = image_path / "train"
test_data_path = image_path / "test"

from torchvision import transforms
data_transform_flip = transforms.Compose([
    transforms.Resize((224,224)),#64,64)),
    # Flip images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5),  # p = probability of flip, 0.5 = 50% chance
    transforms.ToTensor()
])

# If we'd like to make DataLoader's we can now use the function within data_setup.py like so:

from going_modular import data_setup  # Import data_setup.py

# Create train/test dataloader and get class names as a list
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_data_path,
    test_data_path,
    data_transform_flip,
    batch_size=1
)
#    num_workers=2)  # os.cpu_count(), for number of workers per DataLoader
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x79342951b390>,
 ['pizza', 'steak', 'sushi'])

In [7]:
# model_builder.py

%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        # Where did this in_features shape come from?
        # It's because each layer of our network compresses and changes the shape of our inputs data.
        nn.Linear(in_features=hidden_units*56*56, #*13*13,  # =flattened_size
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x
    # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

Writing going_modular/model_builder.py


In [8]:
!ls going_modular/

data_setup.py  get_data.py  model_builder.py  __pycache__


In [9]:
!cat going_modular/model_builder.py

"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
            

In [10]:
import torch
from torch import nn

class TinyVGG2(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        # Where did this in_features shape come from?
        # It's because each layer of our network compresses and changes the shape of our inputs data.
        nn.Linear(in_features=hidden_units*56*56, #*13*13,
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

In [11]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(42)
model_4 = TinyVGG2(input_shape=3,  # number of color channels (3 for RGB)
                   hidden_units=10,
                   output_shape=len(class_names)).to(device)
model_4

TinyVGG2(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=31360, out_features=3, bias=True)
  )
)

In [12]:
# Now instead of coding the TinyVGG model from scratch every time, we can import it using:

import torch
# Import model_builder.py
from going_modular import model_builder

# Create an instance of TinyVGG - Instantiate an instance of the model from the "model_builder.py" script
torch.manual_seed(42)
model_5 = model_builder.TinyVGG(input_shape=3,  # number of color channels (3 for RGB)
                                hidden_units=20,
                                output_shape=len(class_names)).to(device)
model_5

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=62720, out_features=3, bias=True)
  )
)

In [13]:
# engine.py

%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Set up train loss and accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
    # Send data to target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item() / len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)
  return train_loss, train_acc


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Set up test loss and accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Calculate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # Calculate and accumulate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss /= len(dataloader)
  test_acc /= len(dataloader)
  return test_loss, test_acc


def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Sends a target PyTorch model through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    # Print out what's happening
    print(
        f"Epoch: {epoch+1} | "
        f"train_loss: {train_loss:.4f} | "
        f"train_acc: {train_acc:.4f} | "
        f"test_loss: {test_loss:.4f} | "
        f"test_acc: {test_acc:.4f}"
    )

    # Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Writing going_modular/engine.py


In [14]:
from torch import nn
# Set up loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_5.parameters(), lr=0.001)

# Now we've got the engine.py script, we can import functions from it via:

from going_modular import engine

# Use train() by calling it from engine.py
engine.train(model=model_4,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epochs=10,
             device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0989 | train_acc: 0.3444 | test_loss: 1.0966 | test_acc: 0.4200
Epoch: 2 | train_loss: 1.0988 | train_acc: 0.3333 | test_loss: 1.0968 | test_acc: 0.4067
Epoch: 3 | train_loss: 1.0988 | train_acc: 0.3467 | test_loss: 1.0966 | test_acc: 0.4267
Epoch: 4 | train_loss: 1.0989 | train_acc: 0.3311 | test_loss: 1.0967 | test_acc: 0.4000
Epoch: 5 | train_loss: 1.0989 | train_acc: 0.3400 | test_loss: 1.0966 | test_acc: 0.4133
Epoch: 6 | train_loss: 1.0988 | train_acc: 0.3378 | test_loss: 1.0968 | test_acc: 0.4000
Epoch: 7 | train_loss: 1.0988 | train_acc: 0.3422 | test_loss: 1.0967 | test_acc: 0.4200
Epoch: 8 | train_loss: 1.0989 | train_acc: 0.3289 | test_loss: 1.0967 | test_acc: 0.4133
Epoch: 9 | train_loss: 1.0988 | train_acc: 0.3356 | test_loss: 1.0969 | test_acc: 0.3867
Epoch: 10 | train_loss: 1.0988 | train_acc: 0.3422 | test_loss: 1.0968 | test_acc: 0.4000


{'train_loss': [1.0988962247636582,
  1.0988103932804532,
  1.098826891846127,
  1.0988528317875332,
  1.0988813225428264,
  1.098845652209388,
  1.0987908103730943,
  1.098853169017368,
  1.098821231789059,
  1.0988305441538493],
 'train_acc': [0.34444444444444444,
  0.3333333333333333,
  0.3466666666666667,
  0.33111111111111113,
  0.34,
  0.3377777777777778,
  0.3422222222222222,
  0.3288888888888889,
  0.33555555555555555,
  0.3422222222222222],
 'test_loss': [1.0966287493705749,
  1.0968089151382445,
  1.096646505991618,
  1.096662877400716,
  1.0965514810880026,
  1.0967830801010132,
  1.0967257444063823,
  1.096721550623576,
  1.0968641599019369,
  1.0967808556556702],
 'test_acc': [0.42,
  0.4066666666666667,
  0.4266666666666667,
  0.4,
  0.41333333333333333,
  0.4,
  0.42,
  0.41333333333333333,
  0.38666666666666666,
  0.4]}

In [15]:
from going_modular import engine

# Use train() by calling it from engine.py
engine.train(model=model_5,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epochs=15,
             device=device)

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1023 | train_acc: 0.3267 | test_loss: 1.0952 | test_acc: 0.3600
Epoch: 2 | train_loss: 1.0964 | train_acc: 0.3689 | test_loss: 1.0881 | test_acc: 0.4200
Epoch: 3 | train_loss: 1.0892 | train_acc: 0.4267 | test_loss: 1.0756 | test_acc: 0.5533
Epoch: 4 | train_loss: 1.0648 | train_acc: 0.5133 | test_loss: 1.0297 | test_acc: 0.5667
Epoch: 5 | train_loss: 0.9943 | train_acc: 0.5422 | test_loss: 0.9243 | test_acc: 0.5800
Epoch: 6 | train_loss: 0.9181 | train_acc: 0.5689 | test_loss: 0.9034 | test_acc: 0.6000
Epoch: 7 | train_loss: 0.8885 | train_acc: 0.5889 | test_loss: 0.9071 | test_acc: 0.5200
Epoch: 8 | train_loss: 0.8722 | train_acc: 0.5822 | test_loss: 0.8783 | test_acc: 0.6333
Epoch: 9 | train_loss: 0.8523 | train_acc: 0.6089 | test_loss: 0.8792 | test_acc: 0.5533
Epoch: 10 | train_loss: 0.8183 | train_acc: 0.6200 | test_loss: 0.8547 | test_acc: 0.6400
Epoch: 11 | train_loss: 0.8061 | train_acc: 0.6578 | test_loss: 0.8928 | test_acc: 0.5800
Epoch: 12 | train_l

{'train_loss': [1.1023410379886627,
  1.0964280276828342,
  1.0891826815075345,
  1.064801227119234,
  0.9942789374788602,
  0.9181414081570175,
  0.8884713373167648,
  0.8722028453730875,
  0.8523107902084788,
  0.8182701658995615,
  0.8060944353395866,
  0.8038272882687548,
  0.7777070136699411,
  0.7485020143724977,
  0.724616241686874],
 'train_acc': [0.32666666666666666,
  0.3688888888888889,
  0.4266666666666667,
  0.5133333333333333,
  0.5422222222222223,
  0.5688888888888889,
  0.5888888888888889,
  0.5822222222222222,
  0.6088888888888889,
  0.62,
  0.6577777777777778,
  0.6177777777777778,
  0.6688888888888889,
  0.6733333333333333,
  0.7066666666666667],
 'test_loss': [1.095241957505544,
  1.0881380724906922,
  1.0755603071053823,
  1.0297041964530944,
  0.924322037100792,
  0.90342609167099,
  0.9071315926189224,
  0.8782595588080585,
  0.8791625795233995,
  0.8547136928730955,
  0.8928073043158898,
  0.8349447031567494,
  0.8559761411013702,
  0.8415797053424952,
  0.81838

In [16]:
# utils.py

%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create a target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"  # text to display if assert check fails
  model_save_path = target_dir / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


In [17]:
!ls going_modular/


data_setup.py  engine.py  get_data.py  model_builder.py  __pycache__  utils.py


In [18]:
target_dir = Path("saved_models")
model_name = "model_5.pth"

# Now if we wanted to use our save_model() function we can import it and use it via:

from going_modular import utils

# Save a model to file
utils.save_model(model=model_5,
                 target_dir=target_dir,
                 model_name=model_name)

[INFO] Saving model to: saved_models/model_5.pth


In [19]:
# Save a model to file
utils.save_model(model=model_4,
                 target_dir=target_dir,
                 model_name="model_4.pth")

[INFO] Saving model to: saved_models/model_4.pth


In [20]:
# train0.py

%%writefile going_modular/train0.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""
import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms
from pathlib import Path

# Set up hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Set up directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Set up target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=LEARNING_RATE)
#optimizer = torch.optim.Adam(model.parameters(),
#                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(
    model=model,
    target_dir=Path("models"),
    model_name="05_going_modular_script_mode_tinyvgg_model.pth"
)

Writing going_modular/train0.py


In [21]:
# Now we can train a PyTorch model by running the following line on the command line:
!python going_modular/train0.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1023 | train_acc: 0.3208 | test_loss: 1.1003 | test_acc: 0.2875
 20% 1/5 [00:03<00:14,  3.56s/it]Epoch: 2 | train_loss: 1.1011 | train_acc: 0.2958 | test_loss: 1.0975 | test_acc: 0.3500
 40% 2/5 [00:06<00:09,  3.12s/it]Epoch: 3 | train_loss: 1.0998 | train_acc: 0.3250 | test_loss: 1.0966 | test_acc: 0.3500
 60% 3/5 [00:08<00:05,  2.86s/it]Epoch: 4 | train_loss: 1.0995 | train_acc: 0.3292 | test_loss: 1.1007 | test_acc: 0.3500
 80% 4/5 [00:11<00:02,  2.75s/it]Epoch: 5 | train_loss: 1.0993 | train_acc: 0.3521 | test_loss: 1.1027 | test_acc: 0.2875
100% 5/5 [00:14<00:00,  2.81s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


In [22]:
!python going_modular/get_data.py

data/pizza_steak_sushi directory exists.


In [23]:
!ls data/pizza_steak_sushi/test/pizza

1001116.jpg  1618659.jpg  2582289.jpg  3486640.jpg  420409.jpg	771336.jpg
1032754.jpg  1687143.jpg  2782998.jpg  3497151.jpg  441659.jpg	788315.jpg
1067986.jpg  204151.jpg   2901001.jpg  3729167.jpg  44810.jpg	796922.jpg
129666.jpg   2111981.jpg  296426.jpg   3770514.jpg  476421.jpg	833711.jpg
1315645.jpg  2250611.jpg  2997525.jpg  3785667.jpg  482858.jpg	930553.jpg
138961.jpg   2398925.jpg  3174637.jpg  380739.jpg   61656.jpg	998005.jpg
148765.jpg   2549661.jpg  3375083.jpg  416067.jpg   648055.jpg
1555015.jpg  2572488.jpg  3376617.jpg  419962.jpg   724290.jpg


In [24]:
# train.py with argparse

%%writefile train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""
import os
import torch
import argparse

from going_modular import data_setup, engine, model_builder, utils
from torchvision import transforms
from pathlib import Path

# Create a parser
parser = argparse.ArgumentParser(description="Get some hyperparameters.")

# Create an arg for number of epochs
parser.add_argument("--num_epochs",
                    default=10,
                    type=int,
                    help="the number of epochs to train for")

# Create an arg for batch size
parser.add_argument("--batch_size",
                    default=32,
                    type=int,
                    help="number of samples per batch")

# Create an arg for hidden units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help="number of hidden units in hidden layers")

# Create an arg for learning rate
parser.add_argument("--learning_rate",
                    default=0.001,
                    type=float,
                    help="learning rate to use for model")

# Create an arg for training directory
parser.add_argument("--train_dir",
                    default="data/pizza_steak_sushi/train",
                    type=str,
                    help="directory file path to training data in standard image classification format")

# Create an arg for test directory
parser.add_argument("--test_dir",
                    default="data/pizza_steak_sushi/test",
                    type=str,
                    help="directory file path to testing data in standard image classification format")

# Create an arg for model name
parser.add_argument("--model_name",
                    default="model_0.pth",
                    type=str,
                    help="model name to save")

# Get our arguments from the parser
args = parser.parse_args()

# Set up hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[INFO] Training a model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE}, hidden units {HIDDEN_UNITS} and learning rate {LEARNING_RATE}...")

# Set up directories
train_dir = args.train_dir
test_dir = args.test_dir
print(f"[INFO] Training data file: {train_dir}")
print(f"[INFO] Testing data file: {test_dir}")

model_name = args.model_name
print(f"[INFO] Model will be saved as: {model_name}")

# Set up target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=LEARNING_RATE)
#optimizer = torch.optim.Adam(model.parameters(),
#                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
target_dir = Path("saved_models")

utils.save_model(
    model=model,
    target_dir=target_dir,
    model_name=model_name
)

Writing train.py


In [25]:
!python train.py --num_epochs 15 --batch_size 32 --hidden_units 20 --model_name model_1.pth

[INFO] Training a model for 15 epochs with batch size 32, hidden units 20 and learning rate 0.001...
[INFO] Training data file: data/pizza_steak_sushi/train
[INFO] Testing data file: data/pizza_steak_sushi/test
[INFO] Model will be saved as: model_1.pth
  0% 0/15 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.0987 | train_acc: 0.3438 | test_loss: 1.1017 | test_acc: 0.2875
  7% 1/15 [00:03<00:42,  3.05s/it]Epoch: 2 | train_loss: 1.0983 | train_acc: 0.3521 | test_loss: 1.1022 | test_acc: 0.2875
 13% 2/15 [00:05<00:36,  2.81s/it]Epoch: 3 | train_loss: 1.1005 | train_acc: 0.3208 | test_loss: 1.0992 | test_acc: 0.2938
 20% 3/15 [00:09<00:37,  3.15s/it]Epoch: 4 | train_loss: 1.0991 | train_acc: 0.3229 | test_loss: 1.0977 | test_acc: 0.3625
 27% 4/15 [00:11<00:32,  2.98s/it]Epoch: 5 | train_loss: 1.1000 | train_acc: 0.3354 | test_loss: 1.0970 | test_acc: 0.3625
 33% 5/15 [00:14<00:28,  2.86s/it]Epoch: 6 | train_loss: 1.0999 | train_acc: 0.3042 | test_loss: 1.0992 | test_acc: 0.3443
 40% 6/15 [00:17

In [26]:
# Takes too long, too slow with batch_size=64 or 128 and hidden_units=64 or 128
!python train.py --num_epochs 3 --batch_size 64 --hidden_units 16 --learning_rate 0.002 --model_name model_2.pth

[INFO] Training a model for 3 epochs with batch size 64, hidden units 16 and learning rate 0.002...
[INFO] Training data file: data/pizza_steak_sushi/train
[INFO] Testing data file: data/pizza_steak_sushi/test
[INFO] Model will be saved as: model_2.pth
  0% 0/3 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.0977 | train_acc: 0.3613 | test_loss: 1.1148 | test_acc: 0.2396
 33% 1/3 [00:02<00:05,  2.94s/it]Epoch: 2 | train_loss: 1.1067 | train_acc: 0.3008 | test_loss: 1.0910 | test_acc: 0.4583
 67% 2/3 [00:05<00:02,  2.74s/it]Epoch: 3 | train_loss: 1.1034 | train_acc: 0.2930 | test_loss: 1.0989 | test_acc: 0.3021
100% 3/3 [00:08<00:00,  2.71s/it]
[INFO] Saving model to: saved_models/model_2.pth


In [27]:
!ls models
!ls saved_models/

05_going_modular_script_mode_tinyvgg_model.pth
model_1.pth  model_2.pth  model_4.pth  model_5.pth


In [28]:
%%writefile predict.py
"""
Makes predictions with a trained PyTorch model and saves the results to file.
"""
import torch
import torchvision
import argparse
import matplotlib.pyplot as plt

from going_modular import model_builder
from torchvision import transforms

# Create a parser
parser = argparse.ArgumentParser()

# Create an arg for model path
parser.add_argument("--model_path",
                    default="models/05_going_modular_script_mode_tinyvgg_model.pth",
                    type=str,
                    help="filepath of model to use for prediction")

# Create an arg for image path
parser.add_argument("--image_path",
                    default="data/pizza_steak_sushi/train/pizza/12301.jpg",
                    type=str,
                    help="filepath of image to predict on")

# Create an arg for transform type
parser.add_argument("--transform",
                    default="no",
                    type=str,
                    help="yes to transform using horizontal flip, no is default")

# Create an arg for hidden units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help="number of hidden units used by model")

args = parser.parse_args()

model_path = args.model_path
image_path = args.image_path
transform = args.transform
hidden_units = args.hidden_units

print(f"[INFO] Predicting on {image_path} with {model_path}")

# Set up class names
class_names = ["pizza", "steak", "sushi"]

# Set up device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Need to use same hyperparameters as saved model
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=hidden_units,
                              output_shape=3).to(device)  # len(class_names) = 3

# Load in the saved model state dictionary from file
model.load_state_dict(torch.load(model_path))

# 1. Load in an image and convert tensor values to float32
target_image = torchvision.io.read_image(str(image_path)).type(torch.float32)

# 2. Divide the image pixel values by 255 to get them between 0 and 1
target_image /= 255

# 3. Transform if necessary
if transform == "yes":
  data_transform_flip = transforms.Compose([
    transforms.Resize((224,224)),#64,64)),
    # Flip images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5),  # p = probability of flip, 0.5 = 50% chance
    #transforms.ToTensor()
  ])
  target_image = data_transform_flip(target_image)
  print("Using data_transform_flip")
else:  # Resize the image to be the same size as the model
  data_transform = transforms.Compose([
    transforms.Resize((224,224)),#64,64)),
    #transforms.ToTensor()
  ])
  target_image = data_transform(target_image)

# 4. Make sure model is on target device
model.to(device)

# 5. Turn on model evaluation and inference modes
model.eval()
with torch.inference_mode():

  # Add an extra dimension to image
  target_image = target_image.unsqueeze(dim=0)
  # Make a prediction on image with an extra dimension and send it to the target device
  target_image_pred = model(target_image.to(device))

# 6. Convert logits to probabilities
target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

# 7. Convert probs to label
target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

# 8. Plot the image alongside the prediction and prediction probability
plt.imshow(target_image.squeeze().permute(1, 2, 0))  # make sure it's right size for matplotlib
if class_names:
  title = f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu():.3f}"
else:
  title = f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max().cpu():.3f}"
plt.title(title)
plt.axis(False);

print(f"[INFO] Prediction label: {class_names[target_image_pred_label]}, prediction probability: {target_image_pred_probs.max():.3f}")

Writing predict.py


In [29]:
!ls saved_models/

model_1.pth  model_2.pth  model_4.pth  model_5.pth


In [30]:
!python predict.py --image_path data/pizza_steak_sushi/train/pizza/300869.jpg --model_path saved_models/model_5.pth --hidden_units 20 --transform yes
!python predict.py --image_path data/pizza_steak_sushi/train/pizza/300869.jpg --model_path saved_models/model_5.pth --hidden_units 20

[INFO] Predicting on data/pizza_steak_sushi/train/pizza/300869.jpg with saved_models/model_5.pth
Using data_transform_flip
[INFO] Prediction label: steak, prediction probability: 0.584
[INFO] Predicting on data/pizza_steak_sushi/train/pizza/300869.jpg with saved_models/model_5.pth
[INFO] Prediction label: steak, prediction probability: 0.597


In [31]:
!python predict.py --image_path data/pizza_steak_sushi/test/pizza/61656.jpg --model_path saved_models/model_5.pth --hidden_units 20
!python predict.py --image_path data/pizza_steak_sushi/test/pizza/61656.jpg --model_path saved_models/model_5.pth --hidden_units 20 --transform yes

[INFO] Predicting on data/pizza_steak_sushi/test/pizza/61656.jpg with saved_models/model_5.pth
[INFO] Prediction label: sushi, prediction probability: 0.679
[INFO] Predicting on data/pizza_steak_sushi/test/pizza/61656.jpg with saved_models/model_5.pth
Using data_transform_flip
[INFO] Prediction label: sushi, prediction probability: 0.679


In [32]:
# get_custom_data.py

%%writefile going_modular/get_custom_data.py
"""
Contains functionality to download custom images from GitHub
"""
import requests
from pathlib import Path

data_path = Path("data")

# Get multiple custom images
""" to get raw address:  right click jpeg file name in main repository view, select copy link address, paste into browser and enter
    then right click on image, select copy link address = https://github.com/lanehale/pytorch-deep-learning/blob/main/cheese-pizza.jpeg?raw=true
    paste into browser and enter to get url format below
"""
urls = [
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/cheese-pizza.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-slice.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-slice2.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-sliced.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-sliced2.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-partial-view.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-partial-view2.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-side-view.jpeg"
]

filenames = [
    "cheese-pizza.jpeg",
    "pizza-slice.jpeg",
    "pizza-slice2.jpeg",
    "pizza-sliced.jpeg",
    "pizza-sliced2.jpeg",
    "pizza-partial-view.jpeg",
    "pizza-partial-view2.jpeg",
    "pizza-side-view.jpeg"
]

if len(urls) != len(filenames):
  raise ValueError("The number of URLs and filenames must be the same.")

# Download the images if they don't already exist
if (data_path / "cheese-pizza.jpeg").is_file():
  print(f"Custom images already exist, skipping download.")
else:
  for i, url in enumerate(urls):
    try:
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for HTTP errors

      custom_image_path = data_path / filenames[i]

      with open(custom_image_path, "wb") as f:
        f.write(response.content)

      print(f"DownLoading {custom_image_path}...")

    except requests.exceptions.RequestException as e:
      print(f"Error downloading {url}: {e}")
    except Exception as e:
      print(f"An unexpected error occurred: {e}")

Writing going_modular/get_custom_data.py


In [33]:
# Get custom images
!python going_modular/get_custom_data.py

DownLoading data/cheese-pizza.jpeg...
DownLoading data/pizza-slice.jpeg...
DownLoading data/pizza-slice2.jpeg...
DownLoading data/pizza-sliced.jpeg...
DownLoading data/pizza-sliced2.jpeg...
DownLoading data/pizza-partial-view.jpeg...
DownLoading data/pizza-partial-view2.jpeg...
DownLoading data/pizza-side-view.jpeg...


In [34]:
!ls data

cheese-pizza.jpeg	  pizza-side-view.jpeg	pizza-sliced.jpeg
pizza-partial-view2.jpeg  pizza-slice2.jpeg	pizza-slice.jpeg
pizza-partial-view.jpeg   pizza-sliced2.jpeg	pizza_steak_sushi


In [35]:
!python predict.py --image_path data/cheese-pizza.jpeg --model_path saved_models/model_5.pth --hidden_units 20

[INFO] Predicting on data/cheese-pizza.jpeg with saved_models/model_5.pth
[INFO] Prediction label: pizza, prediction probability: 0.579


In [36]:
!python predict.py --image_path data/pizza-sliced.jpeg --model_path saved_models/model_5.pth --hidden_units 20

[INFO] Predicting on data/pizza-sliced.jpeg with saved_models/model_5.pth
[INFO] Prediction label: sushi, prediction probability: 0.631


In [37]:
!python predict.py --image_path data/pizza-partial-view.jpeg --model_path saved_models/model_5.pth --hidden_units 20

[INFO] Predicting on data/pizza-partial-view.jpeg with saved_models/model_5.pth
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.0000001].
[INFO] Prediction label: pizza, prediction probability: 0.714
